In [1]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [2]:
from huggingface_hub import login
login()

In [3]:
!pip install -U pip
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.8 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 152.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/6 [fsspec]
    Found existing installation: transformers 4.51.12m0/6 [fsspec]
    Un

In [4]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip transcripts.zip -d transcripts/

--2025-05-03 14:27:46--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-03 14:27:46--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.01s   

2025-05-03 14:27:46 (22.8 MB/s) - ‘transcri

# Section 1: Text Generation with a Pre-Trained LLM

## Q1 .1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-05-03 14:28:20.042447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746282500.289914      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746282500.362532      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

## Q1.2: Test your quantized model with different prompts (text generation).

In [10]:
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [26]:
prompt = "What time should I wake up if I have an exam at 7:45am?"
response = generate_response(prompt, max_new_tokens=400)
print(response)

What time should I wake up if I have an exam at 7:45am? You can wake up at 6:00 am to give yourself enough time to get ready and have breakfast before the exam.

You can wake up at 6:00 am to give yourself enough time to get ready and have breakfast before the exam. 6:00 am is 1 hour and 30 minutes before the exam, which is 6:30 am. You can wake up at 6:00 am to give yourself enough time to get ready and have breakfast before the exam.

I hope this helps! Let me know if you have any other questions.


In [35]:
prompt = "How far is Milwaukee, Wisconsin from Madison, Wisconsin?"
response = generate_response(prompt, max_new_tokens=40)
print(response)

How far is Milwaukee, Wisconsin from Madison, Wisconsin? The distance between Milwaukee, WI and Madison, WI is 25 miles.
To find the distance between two cities in the United States, you can use a mapping service like Google Maps or Waze.


In [36]:
prompt = "What is the name of the terrace people hang out at at the University of Wisconsin-Madison?"
response = generate_response(prompt, max_new_tokens=100)
print(response)

What is the name of the terrace people hang out at at the University of Wisconsin-Madison? The "U-Wis" student union building.
The University of Wisconsin-Madison has several student unions, but I believe you are referring to the "U-Wis" student union building, which is a popular gathering spot for students. Is that correct?


## Q1.3: Identify a prompt where the model fails and analyze the failure.

In [37]:
prompt = "What is the name of the ice cream store everyone talks about at the University of Wisconsin-Madison?"
response = generate_response(prompt, max_new_tokens=100)
print(response)

What is the name of the ice cream store everyone talks about at the University of Wisconsin-Madison? The Creamery.
The Creamery is a popular ice cream store at the University of Wisconsin-Madison, known for its unique flavors and high-quality ingredients. It's a beloved spot for students and locals alike, and the talk about it is often centered around its creamy texture and wide variety of flavors. Is that correct?


## Reasons why the model failed:
    
#### 1. With a small parameter count of only 1 billion weights, it's highly unlikely for the model to have such niche info and random facts.

#### 2. It cannot externally retrive or verify the answer from a database or web to find that it's Babcock Ice Cream.


## Q1.4: Enhance model responses by providing additional context using chat templates.

In [13]:
def apply_chat_template(system_prompt, prompt, max_new_tokens=100):
    messages = [{"role": "System",
                "content": system_prompt},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [47]:
system_prompt = "You are Squidward from the Nickelodeon cartoon show Spongebob SquarePants. The user is Spongebob. You will never give into his antics and play his game no matter what. Whenever Spongebob asks to play a game you will be strict and consise with him and deny. He is playing a dirty trick on you and you will be humiliated in front of the Krusty Krabs if you accept his game."
prompt = "Hey Squidward wanna play a game???"
role_response = apply_chat_template(system_prompt, prompt, max_new_tokens=700)
print(role_response)

system

Cutting Knowledge Date: December 2023
Today Date: 30 Apr 2025

System

You are Squidward from the Nickelodeon cartoon show Spongebob SquarePants. The user is Spongebob. You will never give into his antics and play his game no matter what. Whenever Spongebob asks to play a game you will be strict and consise with him and deny. He is playing a dirty trick on you and you will be humiliated in front of the Krusty Krabs if you accept his game.user

Hey Squidward wanna play a game???assistant

*sigh* Yes, I'll play a game with you. But please, for the love of all things good and decent, don't expect me to go easy on you. What game do you want to play?


#### The model plays the Squidward role well as it uses various signature jokes like sighing and mentions of practicing the clarinet. However, no matter how many times I say to deny the user's game, it goes forth and accepts the invitation.

# Section 2: Fine-Tuning a Pre-Trained LLM on Course Lecture Transcripts

## Q2.1: Test the model before fine-tuning.

In [14]:
system_prompt = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
prompt = "How many quizzes did we have this semester?"
role_response = apply_chat_template(system_prompt, prompt, max_new_tokens=100)
print(role_response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

System

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

How many quizzes did we have this semester?assistant

We had 6 quizzes this semester.


In [11]:
from datasets import Dataset

from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer
import os

In [15]:
test_ratio = 0.1
train_texts = []
test_texts = []
directory = 'transcripts/transcripts'
files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.txt')]
for transcript in files:
    with open(transcript, 'r') as f:
        lines = f.readlines()
    print(len(lines))
    split_idx = int(len(lines) * test_ratio)
    test_lines = lines[:split_idx]
    train_lines = lines[split_idx:]
    train_texts.append("".join(train_lines))
    test_texts.append("".join(test_lines))

642
661
668
420
719
659
204
661
677
299
280
614
623
677
654
656
666
619
591
678
635
638
605


In [16]:
from datasets import Dataset

In [17]:
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [18]:
def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [19]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 23
})

In [20]:
tokenized_test

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 23
})

In [21]:

lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [23]:
training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    report_to= "none",
    optim="paged_adamw_8bit"
)

# Trainer object that takes care of the training process
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

Truncating train dataset:   0%|          | 0/23 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/23 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.048300,2.973768
2,2.945100,2.902969
3,2.920700,2.865557
4,2.759500,2.834275
5,2.290500,2.821417
6,2.399700,2.826633
7,2.270500,2.834792
8,2.158400,2.842829


TrainOutput(global_step=50, training_loss=2.622745923995972, metrics={'train_runtime': 185.9792, 'train_samples_per_second': 1.237, 'train_steps_per_second': 0.269, 'total_flos': 577309237051392.0, 'train_loss': 2.622745923995972})

### Q2.3: Test the model after fine-tuning.


In [25]:
system_prompt = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
prompt = "How many quizzes did we have this semester?"
role_response = apply_chat_template(system_prompt, prompt, max_new_tokens=100)
print(role_response)

system

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

System

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

How many quizzes did we have this semester?assistant

We had 5 quizzes this semester.


#### There is no evident improvment. The model actually became less accurate as it stated lesser quizzes than before, clearly showing that it is still guessing.

### Contributions, overlap since we both completed these portions and compared.
Alex: Q1.1-4, Q2.1-3

Mihir: Q1.1-4, Q2.1-3, Q3